In [1]:
# from quantitative_analysis import fetch_stock_data, calculate_technical_indicators
import plotly.express as px
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
def fetch_stock_data(ticker:str) -> pd.DataFrame:
    """
    Fetch the 6month stock data of single ticker from yahoo finance.
    """
    stock = yf.download(ticker, period="18mo")
    stock.columns = stock.columns.droplevel(1) # Make the multi-index columns to a single level column; compatible with plotly
    return stock

In [3]:
def calculate_SMA(stock:pd.DataFrame, window:int) -> pd.DataFrame:
    """
    Calculate the SMA (Top Hat Filter) with a given window size of the past 6 months.
    """
    # Get the six_month_ago date: The first date shown on the graph
    six_month_ago = stock.index[-1] - pd.DateOffset(months = 6)
    six_month_ago = stock.index[stock.index > six_month_ago].min().strftime("%Y-%m-%d")

    # Fetch the right amount of data for SMA calculation
    six_month_ago_index = np.where(stock.index == six_month_ago)[0][0]
    stock_for_SMA = stock.iloc[six_month_ago_index-window:].copy()

    SMA = stock_for_SMA["Close"].rolling(window=window).mean()
    stock[f'SMA {window}'] = SMA
    return stock

In [4]:
ticker = 'NVDA'
stock = fetch_stock_data(ticker)
stock = calculate_SMA(stock, 50)
stock = calculate_SMA(stock, 200)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [5]:
# Plot stock price
six_month_ago = stock.index[-1] - pd.DateOffset(months = 6)
six_month_ago = stock.index[stock.index > six_month_ago].min().strftime("%Y-%m-%d")
stock_to_show = stock.loc[stock.index > six_month_ago]
fig = px.line(stock_to_show, 
              x=stock_to_show.index, 
              y=["Close", "SMA 200", "SMA 50"], 
              title=f"{ticker} Stock Price & SMAs")

fig.show()

In [15]:
ticker = 'META'

print('sector', yf.Ticker(ticker).info['sector'])
print('industry', yf.Ticker(ticker).info['industry'])

sector Communication Services
industry Internet Content & Information


In [16]:
yf.Ticker(ticker).info

{'address1': '1 Meta Way',
 'city': 'Menlo Park',
 'state': 'CA',
 'zip': '94025',
 'country': 'United States',
 'phone': '650 543 4800',
 'website': 'https://investor.atmeta.com',
 'industry': 'Internet Content & Information',
 'industryKey': 'internet-content-information',
 'industryDisp': 'Internet Content & Information',
 'sector': 'Communication Services',
 'sectorKey': 'communication-services',
 'sectorDisp': 'Communication Services',
 'longBusinessSummary': 'Meta Platforms, Inc. engages in the development of products that enable people to connect and share with friends and family through mobile devices, personal computers, virtual reality and mixed reality headsets, augmented reality, and wearables worldwide. It operates through two segments, Family of Apps (FoA) and Reality Labs (RL). The FoA segment offers Facebook, which enables people to build community through feed, reels, stories, groups, marketplace, and other; Instagram that brings people closer through instagram feed, s